<a href="https://colab.research.google.com/github/aai540-group3/project/blob/main/bootstrap_aws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bootstrap AWS

In [108]:
%pip install uv --quiet --progress-bar=off
!uv pip install --quiet --system boto3
import os
import textwrap
import boto3
from botocore.exceptions import BotoCoreError, ClientError, ClientError
from google.colab import userdata
from google.colab.userdata import SecretNotFoundError

CONFIG = {
    'AWS_REGION': 'us-west-2',
    'S3_BUCKET_NAME': 'terraform-state-bucket-f3b7a9c1',
    'DYNAMODB_TABLE_NAME': 'terraform-state-lock-e2d8b0a5',
    'CREDENTIALS_FILE': '/root/.aws/credentials',
    'CONFIG_FILE': '/root/.aws/config',
}

def get_aws_credential(credential_name):
    """Retrieve an AWS credential from Google Colab Secrets."""
    try:
        return userdata.get(credential_name)
    except SecretNotFoundError:
        print(f"ERROR: {credential_name} NOT FOUND IN COLAB SECRETS")
        print_secret_setup_instructions(credential_name)
        return None

def print_secret_setup_instructions(secret_name):
    """Print instructions for setting up a secret in Google Colab Secrets UI."""
    print(f"To set '{secret_name}' manually in the Google Colab Secrets UI:")
    print("1. Click the key icon on the left sidebar to open the 'Secrets' pane.")
    print("2. Click the '+' button to add a new secret.")
    print(f"3. Enter '{secret_name}' as the name and your actual AWS credential as the value.")
    print("4. Save the secret and rerun the code cell.")

def ensure_directory_exists(directory):
    """Ensure that the specified directory exists."""
    os.makedirs(directory, exist_ok=True)

def write_file(file_path, content):
    """Write content to a file, creating the directory if it doesn't exist."""
    directory = os.path.dirname(file_path)
    ensure_directory_exists(directory)
    with open(file_path, "w") as f:
        f.write(content)

def write_aws_credentials(aws_access_key_id, aws_secret_access_key, file_path):
    """Write AWS credentials to a file."""
    if not aws_access_key_id or not aws_secret_access_key:
        print("WARNING: AWS CREDENTIALS WRITING SKIPPED - MISSING CREDENTIALS")
        return

    credentials_content = textwrap.dedent(f"""
    [default]
    aws_access_key_id = {aws_access_key_id}
    aws_secret_access_key = {aws_secret_access_key}
    """).strip()
    try:
        write_file(file_path, credentials_content)
        print("STATUS: AWS CREDENTIALS WRITTEN SUCCESSFULLY")
    except Exception as e:
        print(f"ERROR: FAILED TO WRITE AWS CREDENTIALS - {str(e)}")

def write_aws_config(region, file_path):
    """Write AWS config to a file."""
    config_content = textwrap.dedent(f"""
    [default]
    region = {region}
    """).strip()
    try:
        write_file(file_path, config_content)
        print("STATUS: AWS CONFIG WRITTEN SUCCESSFULLY")
    except Exception as e:
        print(f"ERROR: FAILED TO WRITE AWS CONFIG - {str(e)}")

def create_s3_bucket(bucket_name, region):
    """Create an S3 bucket if it doesn't exist"""
    s3_client = boto3.client('s3', region_name=region)
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"STATUS: S3 BUCKET '{bucket_name}' ALREADY EXISTS")
    except ClientError as e:
        if e.response['Error']['Code'] == '404': # Not Found
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
            print(f"STATUS: S3 BUCKET '{bucket_name}' CREATED")
        else:
            print(f"ERROR: FAILED TO CHECK/CREATE S3 BUCKET '{bucket_name}' - {str(e)}")

def create_dynamodb_table(table_name, region):
    """Create a DynamoDB table if it doesn't exist"""
    dynamodb_client = boto3.client('dynamodb', region_name=region)
    try:
        dynamodb_client.describe_table(TableName=table_name)
        print(f"STATUS: DYNAMODB TABLE '{table_name}' ALREADY EXISTS")
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            # Create table
            dynamodb_client.create_table(
                TableName=table_name,
                KeySchema=[
                    {'AttributeName': 'LockID', 'KeyType': 'HASH'}  # Partition key
                ],
                AttributeDefinitions=[
                    {'AttributeName': 'LockID', 'AttributeType': 'S'}
                ],
                BillingMode='PAY_PER_REQUEST'
            )
            print(f"STATUS: DYNAMODB TABLE '{table_name}' CREATED")
        else:
            print(f"ERROR: FAILED TO CHECK/CREATE DYNAMODB TABLE '{table_name}' - {str(e)}")

def bootstrap_aws():
    """Bootstrap AWS environment for Terraform"""
    print("---- STARTING AWS BOOTSTRAP ----")

    # Get AWS credentials from Colab secrets
    aws_access_key_id = get_aws_credential('AWS_ACCESS_KEY_ID')
    aws_secret_access_key = get_aws_credential('AWS_SECRET_ACCESS_KEY')

    # Write AWS credentials and configuration
    write_aws_credentials(aws_access_key_id, aws_secret_access_key, CONFIG['CREDENTIALS_FILE'])
    write_aws_config(CONFIG['AWS_REGION'], CONFIG['CONFIG_FILE'])

    # Check AWS authentication
    if check_aws_authentication():
        # Create S3 bucket and DynamoDB table
        create_s3_bucket(CONFIG['S3_BUCKET_NAME'], CONFIG['AWS_REGION'])
        create_dynamodb_table(CONFIG['DYNAMODB_TABLE_NAME'], CONFIG['AWS_REGION'])

    print("---- AWS BOOTSTRAP COMPLETE ----")

if __name__ == "__main__":
    bootstrap_aws()

---- STARTING AWS BOOTSTRAP ----
STATUS: AWS CREDENTIALS WRITTEN SUCCESSFULLY
STATUS: AWS CONFIG WRITTEN SUCCESSFULLY
STATUS: S3 BUCKET 'terraform-state-bucket-f3b7a9c1' ALREADY EXISTS
STATUS: DYNAMODB TABLE 'terraform-state-lock-e2d8b0a5' ALREADY EXISTS
---- AWS BOOTSTRAP COMPLETE ----
